<a href="https://colab.research.google.com/github/DON2604/20_BIT_CODERS/blob/master/climate_change_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install xarray netCDF4 rasterio shap matplotlib seaborn scikit-learn torch torchvision pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 973.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825

In [6]:
! pip install "cdsapi>=0.7.4"

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Target Features

In [8]:
import kagglehub
import shutil
import os

# downloading from kaggle..........
path = kagglehub.dataset_download("berkeleyearth/climate-change-earth-surface-temperature-data")
print("Downloaded to:", path)

target_dir = "/content/climate_data"
os.makedirs(target_dir, exist_ok=True)

for file_name in os.listdir(path):
    src_file = os.path.join(path, file_name)
    dst_file = os.path.join(target_dir, file_name)
    if os.path.isfile(src_file):
        shutil.copy(src_file, dst_file)

print("Dataset copied to:", target_dir)


Downloaded to: /kaggle/input/climate-change-earth-surface-temperature-data
Dataset copied to: /content/climate_data


In [9]:
%load_ext cudf.pandas
import pandas as pd
import numpy as np
import cudf
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:75: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
/usr/local/lib/python3.11/dist-packages/cudf/pandas/__init__.py:64: UserWarning: Function "cuInit" not found
  warnings.warn(str(e))


In [10]:
df = pd.read_csv("/content/climate_data/GlobalTemperatures.csv")

In [11]:
df.head()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN


# Main Feature

In [12]:
#API creation
%%bash
cat > ~/.cdsapirc <<EOF
url: https://cds.climate.copernicus.eu/api
key: e134c3af-3fa9-430d-84fa-4015ba2675af
EOF

In [14]:
import xarray as xr
import os

import cdsapi

# Downloadinggggggg......

dataset = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": [
        "10m_u_component_of_wind",
        "2m_dewpoint_temperature",
        "boundary_layer_height"
    ],
    "year": ["2011", "2012", "2013"],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "zip"
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()

# Unziiiipinggg........
!unzip /content/8278c562a57ac33c1428531149ba02ac.zip -d /content/reanalysis-era5-single-levels-monthly-means

# Load ERA5 dataset
era5 = xr.open_dataset("/content/reanalysis-era5-single-levels-monthly-means/data_stream-moda_stepType-avgua.nc")
print(era5)

2025-08-03 20:58:34,529 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:ecmwf.datastores.legacy_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-08-03 20:58:35,372 INFO Request ID is b4bb9e2f-b94f-4ac0-82ad-127ce1e17e4a
INFO:ecmwf.datastores.legacy_client:Request ID is b4bb9e2f-b94f-4ac0-82ad-127ce1e17e4a
2025-08-03 20:58:35,630 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-08-03 20:58:41,172 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
2025-08-03 20:58:44,863 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to successful


8278c562a57ac33c1428531149ba02ac.zip:   0%|          | 0.00/168M [00:00<?, ?B/s]

Archive:  /content/8278c562a57ac33c1428531149ba02ac.zip
  inflating: /content/reanalysis-era5-single-levels-monthly-means/data_stream-moda_stepType-avgua.nc  
<xarray.Dataset> Size: 449MB
Dimensions:     (valid_time: 36, latitude: 721, longitude: 1440)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 288B 2011-01-01 ... 2013-12-01
  * latitude    (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    expver      (valid_time) <U4 576B ...
Data variables:
    u10         (valid_time, latitude, longitude) float32 150MB ...
    d2m         (valid_time, latitude, longitude) float32 150MB ...
    blh         (valid_time, latitude, longitude) float32 150MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:

# Static Features

In [15]:
# Elevation (SRTM)
!wget -O /content/elevation.zip "https://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/srtm_39_03.zip"
!unzip /content/elevation.zip -d /content/elevation_data

--2025-08-03 20:59:06--  https://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/srtm_39_03.zip
Resolving srtm.csi.cgiar.org (srtm.csi.cgiar.org)... 45.5.184.25
Connecting to srtm.csi.cgiar.org (srtm.csi.cgiar.org)|45.5.184.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37627843 (36M) [application/zip]
Saving to: ‘/content/elevation.zip’

/content/elevation. 100%[===================>]  35.88M  8.65MB/s    in 4.1s    

2025-08-03 20:59:12 (8.65 MB/s) - ‘/content/elevation.zip’ saved [37627843/37627843]

Archive:  /content/elevation.zip
  inflating: /content/elevation_data/readme.txt  
  inflating: /content/elevation_data/srtm_39_03.hdr  
  inflating: /content/elevation_data/srtm_39_03.tfw  
  inflating: /content/elevation_data/srtm_39_03.tif  


In [16]:
# Land Use (MODIS MCD12Q1) - note: LP DAAC requires Earthdata Login
import os
import requests
from bs4 import BeautifulSoup
import urllib.parse
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

os.makedirs("/content/LandUse", exist_ok=True)

base_url = "https://opendap.cr.usgs.gov/opendap/hyrax/MCD12Q1.061/"

resp = requests.get(base_url + "contents.html")
soup = BeautifulSoup(resp.text, "html.parser")

links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.ncml')]
print(f"Found {len(links)} tiles")

def download_file(link):
    """Download a single ncml file"""
    file_url = urllib.parse.urljoin(base_url, link)
    file_name = os.path.join("/content/LandUse", os.path.basename(link))
    if os.path.exists(file_name):
        return True
    try:
        r = requests.get(file_url, timeout=60)
        r.raise_for_status()
        with open(file_name, "wb") as f:
            f.write(r.content)
        return True
    except:
        return False

max_threads = 10
success_count = 0
fail_count = 0

with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = [executor.submit(download_file, link) for link in links]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Downloading MODIS Tiles"):
        if future.result():
            success_count += 1
        else:
            fail_count += 1

print(f"\n✅ Completed! {success_count} files downloaded successfully, {fail_count} failed.")

Found 315 tiles



✅ Completed! 315 files downloaded successfully, 0 failed.


In [ ]:
# Urban Indicators (SEDAC GRUMPv1)
!unzip /content/CIESIN_SEDAC_GRUMPv1_EXT_1.zip

In [3]:
#!rm -r /content/CIESIN_SEDAC_GRUMPv1_EXT_1